In [1]:
import pathlib
import pickle
import math
import numpy as np
import scipy.sparse
import scipy.io
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.function as fn
from dgl.nn.pytorch import edge_softmax
import time
import argparse
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score

Using backend: pytorch


In [2]:
class edge_coder(nn.Module):
    def __init__(self,in_dim,hidden_dim,out_dim,edge_drop):
        super(edge_coder,self).__init__()
        self.l1=nn.Linear(in_dim,hidden_dim)
        self.l2=nn.Linear(hidden_dim,out_dim)
        self.l3=nn.Linear(hidden_dim,out_dim)
        self.l4=nn.Linear(hidden_dim,out_dim)
        nn.init.xavier_normal_(self.l1.weight,gain=1.414)
    def forward(self,shuru):
        out=self.l1(shuru)
        out=F.relu(out)
   #     out=F.relu(self.l2(out))
     #   out=F.relu(self.l3(out))
     #   out=F.relu(self.l4(out))
    #    out=self.l2(out)
        return out

In [19]:
class aggregation_coder(nn.Module):
    def __init__(self,in_dim,hidden_dim,out_dim):
        super(aggregation_coder,self).__init__()
        self.l1=nn.Linear(in_dim,hidden_dim)
        self.l2=nn.Linear(hidden_dim,out_dim)
        self.l3=nn.Linear(hidden_dim,out_dim)
        self.l4=nn.Linear(hidden_dim,out_dim)
        nn.init.xavier_normal_(self.l1.weight,gain=1.414)
        nn.init.xavier_normal_(self.l2.weight,gain=1.414)
    def forward(self,shuru):
        out=self.l1(shuru)
        out=F.leaky_relu(out)
     #   out=F.leaky_relu(self.l2(out))
     #   out=F.leaky_relu(self.l3(out))
   #     out=F.leaky_relu(self.l4(out))
        return out

In [5]:
class GCN1(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN1, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)
        self.l1=nn.Linear(in_feats,out_feats)
        self.l2=nn.Linear(in_feats,out_feats)
    def message_func(self,edges):
        return {'h':edges.src['h'],'a_drop':edges.data['a_drop'],'aft':edges.data['aft']}
        
    def reduce_func(self,nodes):
        #alpha=F.softmax(nodes.mailbox['e'],dim=1)
        alpha=nodes.mailbox['a_drop']
  #      aft=nodes.mailbox['aft']
     #   h=nodes.mailbox['h']
       # aft=aft.view(-1,64)     
      #  print(alpha.shape)
  #      print(h.shape)
  #      print(aft.shape)
        z=torch.sum(alpha*nodes.mailbox['h']+F.leaky_relu(self.l1(nodes.mailbox['aft'])),dim=1)
     #   z=F.leaky_relu(self.l2(z))
     #   print(z.shape)
     #   z+=aft
        return {'z':z}
    def forward(self, g, feature):
  #      print(g)
        g.ndata['h'] = feature
        g.update_all(self.message_func,self.reduce_func)
        g.apply_nodes(func=self.apply_mod)

        return g.ndata.pop('z')
class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation
        nn.init.xavier_normal_(self.linear.weight,gain=1.414)

    def forward(self, node):
        z= self.linear(node.data['z'])
        if self.activation is not None:
            z = self.activation(z)
    #    z=F.dropout(z, p=0.5, training=self.training)    
        return {'z' : z}

In [6]:
class Net(nn.Module):
    def __init__(self,in_feat,out_feat,activation):
        super(Net,self).__init__()
        self.gcn1=GCN1(in_feat,out_feat,activation)
        self.gcn2=GCN1(in_feat,out_feat,activation)
        self.gcn3=GCN1(in_feat,out_feat,None)
        self.l1=nn.Linear(in_feat,out_feat)
        self.self_weight = nn.Parameter(torch.FloatTensor(in_feat, out_feat))
        self.reset_parameters()
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.self_weight.size(1))
        self.self_weight.data.uniform_(-stdv, stdv)    
       # self.gcn4=GCN2(in_feat,out_feat,activation)
       # self.gcn5=GCN2(in_feat,out_feat,None)
    def forward(self,g,feature):
        x=self.gcn1(g,feature)
    #    x=self.gcn2(g,x)
        y=torch.mm(feature,self.self_weight)
        x+=y
        return x

In [7]:
class MEAHNE_metapath_specific(nn.Module):
    def __init__(self,
                 etypes,
                 out_dim,
                 num_heads,
                 attn_drop=0.5,
                 alpha=0.01,
                 use_minibatch=False,
                 attn_switch=False):
        super(MEAHNE_metapath_specific, self).__init__()
        self.out_dim = out_dim
        self.num_heads = num_heads
       # self.rnn_type = rnn_type
        self.etypes = etypes
        self.use_minibatch = use_minibatch
        self.attn_switch = attn_switch
        self.coder1=edge_coder(out_dim*3,out_dim,out_dim,0.5)
        self.coder2=edge_coder(out_dim*5,out_dim,out_dim,0.5)
        self.coder3=edge_coder(out_dim*4,out_dim,out_dim,0.5)
        self.coder4=edge_coder(out_dim*3,out_dim,out_dim,0.5)
        self.coder5=edge_coder(out_dim*3,out_dim,out_dim,0.5)
        self.coder6=edge_coder(out_dim*4,out_dim,out_dim,0.5)
        self.Net=Net(out_dim,out_dim,F.relu)
       # self.Multi_Net=Multi_Net(out_dim,out_dim,out_dim)
        # rnn-like metapath instance aggregator
        # consider multiple attention heads
        # node-level attention
        # attention considers the center node embedding or not
        if self.attn_switch:
            self.attn1 = nn.Linear(out_dim, num_heads, bias=False)
            self.attn2 = nn.Parameter(torch.empty(size=(1, num_heads, out_dim)))
        else:
            self.attn = nn.Parameter(torch.empty(size=(1, num_heads, out_dim)))
        self.leaky_relu = nn.LeakyReLU(alpha)
        self.softmax = edge_softmax
        if attn_drop:
            self.attn_drop = nn.Dropout(attn_drop)
        else:
            self.attn_drop = lambda x: x

        # weight initialization
        if self.attn_switch:
            nn.init.xavier_normal_(self.attn1.weight, gain=1.414)
            nn.init.xavier_normal_(self.attn2.data, gain=1.414)
        else:
            nn.init.xavier_normal_(self.attn.data, gain=1.414)

    def edge_softmax(self, g):
        attention = self.softmax(g, g.edata.pop('a'))
        # Dropout attention scores and save them
        g.edata['a_drop'] = self.attn_drop(attention)

    def message_passing(self, edges):
        ft = edges.data['aft'] * edges.data['a_drop']
        return {'ft': ft}
    def message_pass(self,edges):
        return {'h':edges.src['h'],'a':edges.data['a']}
    def reduce_func(self,nodes):
        alpha=F.softmax(nodes.mailbox['a'],dim=1)
        alpha_drop=self.attn_drop(alpha)
        z=torch.sum(alpha_drop*nodes.mailbox['h'],dim=1)
        return {'z':z}
    def forward(self, inputs):
        # features: num_all_nodes x out_dim
        if self.use_minibatch:
            g, features, type_mask, edge_metapath_indices, target_idx,nodes,num = inputs
        else:
            g, features, type_mask, edge_metapath_indices,nodes,num= inputs
     #   print(nodes)
   #     print(g)
     #   print(len(nodes))
        g_feat=F.embedding(nodes,features)
      #  print(g_feat.shape)
    #    g.ndata['h']=g_feat
    #    print(g_feat)
        # Embedding layer
        # use torch.nn.functional.embedding or torch.embedding here
        # do not use torch.nn.embedding
        # edata: E x Seq x out_dim
        
        edata = F.embedding(edge_metapath_indices, features)
       # print(edata.shape)
        
        fdata=edata.reshape(edata.shape[0],-1)
     #   print(fdata.shape)
        if(num==1):
            fdata=self.coder1(fdata)
        if(num==2):
            fdata=self.coder2(fdata)
        if(num==3):
            fdata=self.coder3(fdata)
        if(num==4):
            fdata=self.coder4(fdata)
        if(num==5):
            fdata=self.coder5(fdata)
        if(num==6):
            fdata=self.coder6(fdata)
        
        #eft = hidden.permute(1, 0, 2).view(-1, self.num_heads, self.out_dim)  # E x num_heads x out_dim
   #     print(self.attn.shape)
    #    print(fdata.shape)
        aft=fdata.unsqueeze(1)
   #     print(aft.shape)
        a = (aft * self.attn).sum(dim=-1)
       
       # a = (aft * self.attn).sum(dim=-1).unsqueeze(dim=-1)  # E x num_heads x 1
   #     print('ashape')
   #     print(a.shape)    
     #   a = self.leaky_relu(a)
        aft=aft.squeeze(dim=1)
    #    print(aft.shape)
        g.edata.update({'aft': aft, 'a': a})
        # compute softmax normalized attention values
        self.edge_softmax(g)
        # compute the aggregated node features scaled by the dropped,
        # unnormalized attention values.
        #g.update_all(self.message_passing, fn.sum('ft', 'ft'))
 #       g.update_all(self.message_pass,self.reduce_func)
  #      ret=g.ndata.pop('z')
   #     ret = g.ndata.pop('ft')  # E x num_heads x out_dim
        ret=self.Net(g,g_feat)
        if self.use_minibatch:
            return ret[target_idx]
        else:
            return ret


class MEAHNE_ctr_ntype_specific(nn.Module):
    def __init__(self,
                 num_metapaths,
                 etypes_list,
                 out_dim,
                 num_heads,
                 attn_vec_dim,
                 attn_drop=0.5,
                 use_minibatch=False):
        super(MEAHNE_ctr_ntype_specific, self).__init__()
        self.out_dim = out_dim
        self.num_heads = num_heads
        self.use_minibatch = use_minibatch
        self.emb=[]
        # metapath-specific layers
        self.metapath_layers = nn.ModuleList()
        self.aggregation_coder=aggregation_coder(out_dim*3,out_dim,out_dim)
        for i in range(num_metapaths):
            self.metapath_layers.append(MEAHNE_metapath_specific(etypes_list[i],
                                                                out_dim,
                                                                num_heads,
                                                                attn_drop=attn_drop,
                                                                use_minibatch=use_minibatch))

        # metapath-level attention
        # note that the acutal input dimension should consider the number of heads
        # as multiple head outputs are concatenated together
        self.fc1 = nn.Linear(out_dim * num_heads, attn_vec_dim, bias=True)
        self.fc2 = nn.Linear(attn_vec_dim, 1, bias=False)
        self.drop10=nn.Linear(out_dim*num_heads,attn_vec_dim,bias=True)
        self.drop11=nn.Linear(attn_vec_dim, 1, bias=False)
        self.drop20=nn.Linear(out_dim*num_heads,attn_vec_dim,bias=True)
        self.drop21=nn.Linear(attn_vec_dim, 1, bias=False)
        self.drop30=nn.Linear(out_dim*num_heads,attn_vec_dim,bias=True)
        self.drop31=nn.Linear(attn_vec_dim, 1, bias=False)
        
        
        # weight initialization
        nn.init.xavier_normal_(self.fc1.weight, gain=1.414)
        nn.init.xavier_normal_(self.fc2.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop10.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop11.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop20.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop21.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop30.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop31.weight, gain=1.414)
        
    def get_loss(self):
        loss=0
        emb1=self.emb[0]
        emb2=self.emb[1]
        emb3=self.emb[2]
        loss+=torch.mm(emb1,emb2.T)
        loss+=torch.mm(emb1,emb3.T)
        loss+=torch.mm(emb2,emb3.T)
        return loss 
    
    def forward(self, inputs):
   
        if self.use_minibatch:
            g_list, features, type_mask, edge_metapath_indices_list, target_idx_list,node_list,num_list = inputs
            # metapath-specific layers
            metapath_outs = [F.elu(metapath_layer((g, features, type_mask, edge_metapath_indices, target_idx,nodes,num)).view(-1, self.num_heads * self.out_dim))
                             for g, edge_metapath_indices, target_idx, nodes,num,metapath_layer in zip(g_list, edge_metapath_indices_list, target_idx_list,node_list,num_list,self.metapath_layers)]
        else:
            g_list, features, type_mask, edge_metapath_indices_list,node_list,num_list = inputs

            # metapath-specific layers
            metapath_outs = [F.elu(metapath_layer((g, features, type_mask, edge_metapath_indices)).view(-1, self.num_heads * self.out_dim))
                             for g, edge_metapath_indices, metapath_layer in zip(g_list, edge_metapath_indices_list, self.metapath_layers)]
        ceta=torch.cat(metapath_outs,dim=1)
        h=self.aggregation_coder(ceta)
        return h

In [8]:
class MEAHNE_lp_layer(nn.Module):
    def __init__(self,
                 num_metapaths_list,
                 num_edge_type,
                 etypes_lists,
                 in_dim,
                 out_dim,
                 num_heads,
                 attn_vec_dim,
                 attn_drop=0.5):
        super(MEAHNE_lp_layer, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.num_heads = num_heads
        # ctr_ntype-specific layers
        self.user_layer = MEAHNE_ctr_ntype_specific(num_metapaths_list[0],
                                                   etypes_lists[0],
                                                   in_dim,
                                                   num_heads,
                                                   attn_vec_dim,
                                                   attn_drop,
                                                   use_minibatch=True)
        self.item_layer = MEAHNE_ctr_ntype_specific(num_metapaths_list[1],
                                                   etypes_lists[1],
                                                   in_dim,
                                                   num_heads,
                                                   attn_vec_dim,
                                                   attn_drop,
                                                   use_minibatch=True)

        # note that the acutal input dimension should consider the number of heads
        # as multiple head outputs are concatenated together
        self.fc_user = nn.Linear(in_dim * num_heads, out_dim, bias=True)
        self.fc_item = nn.Linear(in_dim * num_heads, out_dim, bias=True)
        nn.init.xavier_normal_(self.fc_user.weight, gain=1.414)
        nn.init.xavier_normal_(self.fc_item.weight, gain=1.414)
    def get_loss(self):
        loss=0
        loss+=self.user_layer.get_loss()
        loss+=self.item_layer.get_loss()
        return loss
    def forward(self, inputs):
        g_lists, features, type_mask, edge_metapath_indices_lists, target_idx_lists,node_lists= inputs

        # ctr_ntype-specific layers
        h_user = self.user_layer(
            (g_lists[0], features, type_mask, edge_metapath_indices_lists[0], target_idx_lists[0],node_lists[0],[1,2,3]))

        h_item = self.item_layer(
            (g_lists[1], features, type_mask, edge_metapath_indices_lists[1], target_idx_lists[1],node_lists[1],[4,5,6]))

        logits_user = self.fc_user(h_user)
        logits_item = self.fc_item(h_item)
        return [logits_user, logits_item], [h_user, h_item]


class MEAHNE_lp(nn.Module):
    def __init__(self,
                 num_metapaths_list,
                 num_edge_type,
                 etypes_lists,
                 feats_dim_list,
                 hidden_dim,
                 out_dim,
                 num_heads,
                 attn_vec_dim,
                 dropout_rate=0.5):
        super(MEAHNE_lp, self).__init__()
        self.hidden_dim = hidden_dim

        # ntype-specific transformation
        self.fc_list = nn.ModuleList([nn.Linear(feats_dim, hidden_dim, bias=True) for feats_dim in feats_dim_list])
        # feature dropout after trainsformation
        if dropout_rate > 0:
            self.feat_drop = nn.Dropout(dropout_rate)
        else:
            self.feat_drop = lambda x: x
        # initialization of fc layers
        for fc in self.fc_list:
            nn.init.xavier_normal_(fc.weight, gain=1.414)

        # MEAHNE_lp layers
        self.layer1 = MEAHNE_lp_layer(num_metapaths_list,
                                     num_edge_type,
                                     etypes_lists,
                                     hidden_dim,
                                     out_dim,
                                     num_heads,
                                     attn_vec_dim,
                                     attn_drop=dropout_rate)
    def get_loss(self):
        return self.layer1.get_loss()
    def forward(self, inputs):
        g_lists, features_list, type_mask, edge_metapath_indices_lists, target_idx_lists,node_lists = inputs

        # ntype-specific transformation
        transformed_features = torch.zeros(type_mask.shape[0], self.hidden_dim, device=features_list[0].device)
        for i, fc in enumerate(self.fc_list):
            node_indices = np.where(type_mask == i)[0]
            transformed_features[node_indices] = fc(features_list[i])
        transformed_features = self.feat_drop(transformed_features)

        # hidden layers
        [logits_user, logits_item], [h_user, h_item] = self.layer1(
            (g_lists, transformed_features, type_mask, edge_metapath_indices_lists, target_idx_lists,node_lists))

        return [logits_user, logits_item], [h_user, h_item]


In [9]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience, verbose=False, delta=0, save_path='checkpoint.pt'):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.save_path = save_path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score - self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
            with open(f,"a") as file:
                file.write('Validation loss decreased :'+str(self.val_loss_min)+'-->'+str(val_loss)+'Saving model ...'+"\n") 
        torch.save(model.state_dict(), self.save_path)
        self.val_loss_min = val_loss


In [10]:
def load_LastFM_data(prefix='guoqing_data/'):
    in_file = open(prefix + '0/0-1-0.adjlist', 'r')
    adjlist00 = [line.strip() for line in in_file]
    adjlist00 = adjlist00
    in_file.close()
    in_file = open(prefix + '0/0-1-2-1-0.adjlist', 'r')
    adjlist01 = [line.strip() for line in in_file]
    adjlist01 = adjlist01
    in_file.close()
    in_file = open(prefix + '0/0-2-2-0.adjlist', 'r')
    adjlist02 = [line.strip() for line in in_file]
    adjlist02 = adjlist02
    in_file.close()
    in_file = open(prefix + '1/1-0-1.adjlist', 'r')
    adjlist10 = [line.strip() for line in in_file]
    adjlist10 = adjlist10
    in_file.close()
    in_file = open(prefix + '1/1-2-1.adjlist', 'r')
    adjlist11 = [line.strip() for line in in_file]
    adjlist11 = adjlist11
    in_file.close()
    in_file = open(prefix + '1/1-2-2-1.adjlist', 'r')
    adjlist12 = [line.strip() for line in in_file]
    adjlist12 = adjlist12
    in_file.close()

    in_file = open(prefix + '0/0-1-0_idx.pickle', 'rb')
    idx00 = pickle.load(in_file)
    in_file.close()
    in_file = open(prefix + '0/0-1-2-1-0_idx.pickle', 'rb')
    idx01 = pickle.load(in_file)
    in_file.close()
    in_file = open(prefix + '0/0-2-2-0_idx.pickle', 'rb')
    idx02 = pickle.load(in_file)
    in_file.close()
    in_file = open(prefix + '1/1-0-1_idx.pickle', 'rb')
    idx10 = pickle.load(in_file)
    in_file.close()
    in_file = open(prefix + '1/1-2-1_idx.pickle', 'rb')
    idx11 = pickle.load(in_file)
    in_file.close()
    in_file = open(prefix + '1/1-2-2-1_idx.pickle', 'rb')
    idx12 = pickle.load(in_file)
    in_file.close()

    adjM = scipy.sparse.load_npz(prefix + 'adjM.npz')
    type_mask = np.load(prefix + 'node_types.npy')
    train_val_test_pos_mir_disease = np.load(prefix + '73clear_train_val_test_pos_mir_disease.npz')
    train_val_test_neg_mir_disease = np.load(prefix + '73clear_train_val_test_neg_mir_disease.npz')

    return [[adjlist00, adjlist01,adjlist02],[adjlist10, adjlist11,adjlist12]],\
           [[idx00, idx01,idx02], [idx10, idx11,idx12]],\
           adjM, type_mask, train_val_test_pos_mir_disease, train_val_test_neg_mir_disease


In [11]:
def parse_adjlist_LastFM(adjlist, edge_metapath_indices, samples=None, exclude=None, offset=None, mode=None):
    edges = []
    nodes = set()
    result_indices = []
    #print(offset)
    #print(exclude)
    for row, indices in zip(adjlist, edge_metapath_indices):
        row_parsed = list(map(int, row.split(' ')))
        nodes.add(row_parsed[0])
        if len(row_parsed) > 1:
            # sampling neighbors
            if samples is None:
                if exclude is not None:
                    if mode == 0:
                        mask = [False if [u1, a1 - offset] in exclude or [u2, a2 - offset] in exclude else True for u1, a1, u2, a2 in indices[:, [0, 1, -1, -2]]]
                    else:
                        mask = [False if [u1, a1 - offset] in exclude or [u2, a2 - offset] in exclude else True for a1, u1, a2, u2 in indices[:, [0, 1, -1, -2]]]
                    neighbors = np.array(row_parsed[1:])[mask]
                    result_indices.append(indices[mask])
                else:
                    neighbors = row_parsed[1:]
                    result_indices.append(indices)
            else:
                # undersampling frequent neighbors
                unique, counts = np.unique(row_parsed[1:], return_counts=True)
                p = []
                for count in counts:
                    p += [(count ** (3 / 4)) / count] * count
                p = np.array(p)
                p = p / p.sum()
                samples = min(samples, len(row_parsed) - 1)
                sampled_idx = np.sort(np.random.choice(len(row_parsed) - 1, samples, replace=False, p=p))
                if exclude is not None:
                    if mode == 0:
                        mask = [False if [u1, a1 - offset] in exclude or [u2, a2 - offset] in exclude else True for u1, a1, u2, a2 in indices[sampled_idx][:, [0, 1, -1, -2]]]
                    else:
                        mask = [False if [u1, a1 - offset] in exclude or [u2, a2 - offset] in exclude else True for a1, u1, a2, u2 in indices[sampled_idx][:, [0, 1, -1, -2]]]
                    neighbors = np.array([row_parsed[i + 1] for i in sampled_idx])[mask]
                    result_indices.append(indices[sampled_idx][mask])
                else:
                    neighbors = [row_parsed[i + 1] for i in sampled_idx]
                    result_indices.append(indices[sampled_idx])
        else:
            neighbors = [row_parsed[0]]
            indices = np.array([[row_parsed[0]] * indices.shape[1]])
            if mode == 1:
                indices += offset
            result_indices.append(indices)
        for dst in neighbors:
            nodes.add(dst)
            edges.append((row_parsed[0], dst))
    nodes=sorted(nodes)    
   # print(nodes)
    if mode==1:
        node_list=[offset+i for i in nodes]
    else:
        node_list=[i for i in nodes]
 #   print(node_list)    
    mapping = {map_from: map_to for map_to, map_from in enumerate(sorted(nodes))}
    edges = list(map(lambda tup: (mapping[tup[0]], mapping[tup[1]]), edges))
    result_indices = np.vstack(result_indices)
    return edges, result_indices, len(nodes), mapping,node_list


def parse_minibatch_LastFM(adjlists_ua, edge_metapath_indices_list_ua, user_artist_batch, device, samples=None, use_masks=None, offset=None):
    g_lists = [[], []]
    result_indices_lists = [[], []]
    idx_batch_mapped_lists = [[], []]
    node_list=[[],[]]
    for mode, (adjlists, edge_metapath_indices_list) in enumerate(zip(adjlists_ua, edge_metapath_indices_list_ua)):
        for adjlist, indices, use_mask in zip(adjlists, edge_metapath_indices_list, use_masks[mode]):
     #       print(offset)
            if use_mask:
                edges, result_indices, num_nodes, mapping,nodes = parse_adjlist_LastFM(
                    [adjlist[row[mode]] for row in user_artist_batch], [indices[row[mode]] for row in user_artist_batch], samples, user_artist_batch, offset, mode)
            else:
                edges, result_indices, num_nodes, mapping,nodes = parse_adjlist_LastFM(
                    [adjlist[row[mode]] for row in user_artist_batch], [indices[row[mode]] for row in user_artist_batch], samples, offset=offset, mode=mode)
            
         #   num_edges=len(edges)
            
        #    print(edges)
       #     print(mapping)
      #      z=np.random.choice(np.arange(num_edges),int(0.9*num_edges),replace=False)
      #      d_edges=[edges[i] for i in z]
      #      result_indices=result_indices[z]
            d_edges=edges
            g = dgl.DGLGraph(multigraph=True)
            g.add_nodes(num_nodes)                
            if len(d_edges)>0:
                sorted_index=sorted(range(len(d_edges)),key=lambda i :d_edges[i])
                g.add_edges(*list(zip(*[(d_edges[i][1],d_edges[i][0]) for i in sorted_index]))) 
                d_result_indices = torch.LongTensor(result_indices[sorted_index]).to(device)
                    
            else:
                d_result_indices=torch.LongTensor(result_indices).to(device)
            g=g.to(device)
            g_lists[mode].append(g)
            result_indices_lists[mode].append(d_result_indices)
            idx_batch_mapped_lists[mode].append(np.array([mapping[row[mode]] for row in user_artist_batch]))       
            nodes=torch.tensor(nodes).to(device)
   #         print(mode)
   #         print(nodes)
            node_list[mode].append(nodes)
    return g_lists, result_indices_lists, idx_batch_mapped_lists,node_list


class index_generator:
    def __init__(self, batch_size, num_data=None, indices=None, shuffle=True):
        if num_data is not None:
            self.num_data = num_data
            self.indices = np.arange(num_data)
        if indices is not None:
            self.num_data = len(indices)
            self.indices = np.copy(indices)
        self.batch_size = batch_size
        self.iter_counter = 0
        self.shuffle = shuffle
        if shuffle:
            np.random.shuffle(self.indices)

    def next(self):
        if self.num_iterations_left() <= 0:
            self.reset()
        self.iter_counter += 1
        return np.copy(self.indices[(self.iter_counter - 1) * self.batch_size:self.iter_counter * self.batch_size])

    def num_iterations(self):
        return int(np.ceil(self.num_data / self.batch_size))

    def num_iterations_left(self):
        return self.num_iterations() - self.iter_counter

    def reset(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
        self.iter_counter = 0


In [20]:
num_ntype = 3
dropout_rate = 0.5
lr = 0.005
weight_decay = 0.001
#etypes_lists = [[[0, 1],[0, 2, 3, 1]],
 #               [[1, 0],[2, 3]]
etypes_lists = [[[0, 1],[0, 2, 3, 1],[4,None,5]],
                [[1, 0],[2, 3],[2, None, 3]]]
use_masks = [[True, True],
             [True, False]]
no_masks = [[False] * 3, [False] * 3]
#use_masks = [[True, True, False],
#             [True, False, True]]
#no_masks = [[False] * 3, [False] * 3]
num_mir=1296
num_disease=11783
num_gene=10116
expected_metapaths = [
   [(0, 1, 0), (0, 1, 2, 1, 0), (0,2,2,0)],
    [(1, 0, 1), (1, 2, 1), (1, 2, 2, 1)]
]
f='0727_5.txt'

def run_model_OURS(feats_type, hidden_dim, num_heads, attn_vec_dim,
                     num_epochs, patience, batch_size, neighbor_samples, repeat,save_postfix):
    adjlists_ua, edge_metapath_indices_list_ua, _, type_mask, train_val_test_pos_user_artist, train_val_test_neg_user_artist = load_LastFM_data()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    features_list = []
    in_dims = []
    if feats_type == 0:
        for i in range(num_ntype):
            dim = (type_mask == i).sum()
            in_dims.append(dim)
            indices = np.vstack((np.arange(dim), np.arange(dim)))
            indices = torch.LongTensor(indices)
            values = torch.FloatTensor(np.ones(dim))
            features_list.append(torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device))
    elif feats_type == 1:
        for i in range(num_ntype):
            dim = 10
            num_nodes = (type_mask == i).sum()
            in_dims.append(dim)
            features_list.append(torch.zeros((num_nodes, 10)).to(device))
    train_pos_user_artist = train_val_test_pos_user_artist['train_pos_mir_disease']
    val_pos_user_artist = train_val_test_pos_user_artist['val_pos_mir_disease']
    test_pos_user_artist = train_val_test_pos_user_artist['test_pos_mir_disease']
    train_neg_user_artist = train_val_test_neg_user_artist['train_neg_mir_disease']
    val_neg_user_artist = train_val_test_neg_user_artist['val_neg_mir_disease']
    test_neg_user_artist = train_val_test_neg_user_artist['test_neg_mir_disease']
    y_true_test = np.array([1] * len(test_pos_user_artist) + [0] * len(test_neg_user_artist))

    auc_list = []
    ap_list = []

    with open(f,"a") as file:
        file.write('单层dropout0.6+torch.mm(features,self.weight)'+"\n")
    for _ in range(repeat):
        net = MEAHNE_lp(
            [3, 3], 6, etypes_lists, in_dims, hidden_dim, hidden_dim, num_heads, attn_vec_dim, dropout_rate)
        net.to(device)
        optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

        # training loop
        net.train()
        early_stopping = EarlyStopping(patience=patience, verbose=True, save_path='checkpoint/checkpoint_{}.pt'.format(save_postfix))
        dur1 = []
        dur2 = []
        dur3 = []
        train_pos_idx_generator = index_generator(batch_size=batch_size, num_data=len(train_pos_user_artist))
        val_idx_generator = index_generator(batch_size=batch_size, num_data=len(val_pos_user_artist), shuffle=False)
        for epoch in range(num_epochs):
            t_start = time.time()
            # training
            net.train()
            for iteration in range(train_pos_idx_generator.num_iterations()):
                # forward
                t0 = time.time()

                train_pos_idx_batch = train_pos_idx_generator.next()
                train_pos_idx_batch.sort()
               
                train_pos_user_artist_batch = train_pos_user_artist[train_pos_idx_batch].tolist()
                train_neg_idx_batch = np.random.choice(len(train_neg_user_artist), len(train_pos_idx_batch))
                train_neg_idx_batch.sort()
                train_neg_user_artist_batch = train_neg_user_artist[train_neg_idx_batch].tolist()
                
                #shuffle
                num_pos = train_pos_idx_batch.shape[0]
                train_batch = np.concatenate([train_pos_user_artist_batch, train_neg_user_artist_batch], axis=0)
                y_label = np.zeros((train_batch.shape[0], 1), dtype=int)
                y_label[:num_pos] = 1
                train_data = np.concatenate([train_batch, y_label], axis=1)
                np.random.shuffle(train_data)
                train_batch = train_data[:, :-1]
                y_label = train_data[:, -1]

                train_g_lists, train_indices_lists, train_idx_batch_mapped_lists ,node_lists= parse_minibatch_LastFM(
                   adjlists_ua, edge_metapath_indices_list_ua, train_batch, device, neighbor_samples, no_masks, num_mir)
                t1 = time.time()
                dur1.append(t1 - t0)

                [embedding_user, embedding_artist], _ = net(
                    (train_g_lists, features_list, type_mask, train_indices_lists, train_idx_batch_mapped_lists,node_lists))
                
                embedding_user = embedding_user.view(-1, 1, embedding_user.shape[1])
                embedding_artist = embedding_artist.view(-1, embedding_artist.shape[1], 1)
                
                out = torch.bmm(embedding_user, embedding_artist)
                class_op = torch.LongTensor([1 if l == 1 else -1 for l in y_label]).view(-1, 1, 1).to(device)
                
                train_loss = -torch.mean(F.logsigmoid(out * class_op))#-torch.mean(F.logsigmoid(net.get_loss()*0.5))
                
                t2 = time.time()
                dur2.append(t2 - t1)

                # autograd
                optimizer.zero_grad()
                train_loss.backward()
                optimizer.step()

                t3 = time.time()
                dur3.append(t3 - t2)

                # print training info
                if iteration % 100 == 0:
                    print(
                        'Epoch {:05d} | Iteration {:05d} | Train_Loss {:.4f} | Time1(s) {:.4f} | Time2(s) {:.4f} | Time3(s) {:.4f}'.format(
                            epoch, iteration, train_loss.item(), np.mean(dur1), np.mean(dur2), np.mean(dur3)))
                    with open(f,"a") as file:
                        file.write('epoch:'+str(epoch)+'iteration:'+str(iteration)+'train_loss:'+str(train_loss.item())+"time1:"+ str(np.mean(dur1))+'time2'
                               +str(np.mean(dur2))+'time3'+str(np.mean(dur3))+"\n")    
            # validation
            net.eval()
            val_loss = []
            with torch.no_grad():
                for iteration in range(val_idx_generator.num_iterations()):
                    # forward
                    val_idx_batch = val_idx_generator.next()
                    val_pos_user_artist_batch = val_pos_user_artist[val_idx_batch].tolist()
                    val_neg_user_artist_batch = val_neg_user_artist[val_idx_batch].tolist()
                    val_pos_g_lists, val_pos_indices_lists, val_pos_idx_batch_mapped_lists,val_pos_node_lists = parse_minibatch_LastFM(
                        adjlists_ua, edge_metapath_indices_list_ua, val_pos_user_artist_batch, device, neighbor_samples, no_masks, num_mir)
                    val_neg_g_lists, val_neg_indices_lists, val_neg_idx_batch_mapped_lists,val_neg_node_lists= parse_minibatch_LastFM(
                        adjlists_ua, edge_metapath_indices_list_ua, val_neg_user_artist_batch, device, neighbor_samples, no_masks, num_mir)

                    [pos_embedding_user, pos_embedding_artist], _ = net(
                        (val_pos_g_lists, features_list, type_mask, val_pos_indices_lists, val_pos_idx_batch_mapped_lists,val_pos_node_lists))
                    [neg_embedding_user, neg_embedding_artist], _ = net(
                        (val_neg_g_lists, features_list, type_mask, val_neg_indices_lists, val_neg_idx_batch_mapped_lists,val_neg_node_lists))
                    pos_embedding_user = pos_embedding_user.view(-1, 1, pos_embedding_user.shape[1])
                    pos_embedding_artist = pos_embedding_artist.view(-1, pos_embedding_artist.shape[1], 1)
                    neg_embedding_user = neg_embedding_user.view(-1, 1, neg_embedding_user.shape[1])
                    neg_embedding_artist = neg_embedding_artist.view(-1, neg_embedding_artist.shape[1], 1)

                    pos_out = torch.bmm(pos_embedding_user, pos_embedding_artist)
                    neg_out = -torch.bmm(neg_embedding_user, neg_embedding_artist)
                    val_loss.append(-torch.mean(F.logsigmoid(pos_out) + F.logsigmoid(neg_out)))#-torch.mean(F.logsigmoid(net.get_loss()*0.5)))
                val_loss = torch.mean(torch.tensor(val_loss))
            t_end = time.time()
            # print validation info
            print('Epoch {:05d} | Val_Loss {:.4f} | Time(s) {:.4f}'.format(
                epoch, val_loss.item(), t_end - t_start))
            with open(f,"a") as file:
                file.write('epoch:'+str(epoch)+'val_loss:'+str(val_loss.item())+"time:"+str(t_end-t_start)+"\n")
            # early stopping
            early_stopping(val_loss, net)
            if early_stopping.early_stop:
                print('Early stopping!')
                with open(f,"a") as file:
                    file.write('epoch:'+str(epoch)+'early stopping!'+"\n")
                break
            
        test_idx_generator = index_generator(batch_size=batch_size, num_data=len(test_pos_user_artist), shuffle=False)
        net.load_state_dict(torch.load('checkpoint/checkpoint_{}.pt'.format(save_postfix)))
        net.eval()
        pos_proba_list = []
        neg_proba_list = []
        with torch.no_grad():
            for iteration in range(test_idx_generator.num_iterations()):
                # forward
                test_idx_batch = test_idx_generator.next()
                test_pos_user_artist_batch = test_pos_user_artist[test_idx_batch].tolist()
                test_neg_user_artist_batch = test_neg_user_artist[test_idx_batch].tolist()
                test_pos_g_lists, test_pos_indices_lists, test_pos_idx_batch_mapped_lists,test_pos_node_lists = parse_minibatch_LastFM(
                    adjlists_ua, edge_metapath_indices_list_ua, test_pos_user_artist_batch, device, neighbor_samples, no_masks, num_mir)
                test_neg_g_lists, test_neg_indices_lists, test_neg_idx_batch_mapped_lists,test_neg_node_lists = parse_minibatch_LastFM(
                    adjlists_ua, edge_metapath_indices_list_ua, test_neg_user_artist_batch, device, neighbor_samples, no_masks, num_mir)

                [pos_embedding_user, pos_embedding_artist], _ = net(
                    (test_pos_g_lists, features_list, type_mask, test_pos_indices_lists, test_pos_idx_batch_mapped_lists,test_pos_node_lists))
                
                [neg_embedding_user, neg_embedding_artist], _ = net(
                    (test_neg_g_lists, features_list, type_mask, test_neg_indices_lists, test_neg_idx_batch_mapped_lists,test_neg_node_lists))
                pos_embedding_user = pos_embedding_user.view(-1, 1, pos_embedding_user.shape[1])
                pos_embedding_artist = pos_embedding_artist.view(-1, pos_embedding_artist.shape[1], 1)
                neg_embedding_user = neg_embedding_user.view(-1, 1, neg_embedding_user.shape[1])
                neg_embedding_artist = neg_embedding_artist.view(-1, neg_embedding_artist.shape[1], 1)

                pos_out = torch.bmm(pos_embedding_user, pos_embedding_artist).flatten()
                neg_out = torch.bmm(neg_embedding_user, neg_embedding_artist).flatten()
                pos_proba_list.append(torch.sigmoid(pos_out))
                neg_proba_list.append(torch.sigmoid(neg_out))
            y_proba_test = torch.cat(pos_proba_list + neg_proba_list)
            y_proba_test = y_proba_test.cpu().numpy()
        np.savez('OURS_prediction_result.npz', y_true=y_true_test,
                y_pred=y_proba_test)    
        auc = roc_auc_score(y_true_test, y_proba_test)
        ap = average_precision_score(y_true_test, y_proba_test)
        print('Link Prediction Test')
        print('AUC = {}'.format(auc))
        print('AP = {}'.format(ap))
        auc_list.append(auc)
        ap_list.append(ap)
        with open(f,"a") as file:
                file.write('Link Prediction Test:'+"\n"+'AUC = {}:'+str(auc)+"|"+"AP = {}:"+str(ap)+"\n")
    print('----------------------------------------------------------------')
    print('Link Prediction Tests Summary')
    print('AUC_mean = {}, AUC_std = {}'.format(np.mean(auc_list), np.std(auc_list)))
    print('AP_mean = {}, AP_std = {}'.format(np.mean(ap_list), np.std(ap_list)))
    with open(f,"a") as file:
        file.write('Link Prediction Tests Summary:'+"\n"+"AUC_mean = {}"+str(np.mean(auc_list))+'AUC_std = {}'+str(np.std(auc_list))+
                           "\n"+'AP_mean = {} '+str(np.mean(ap_list))+'AP_std = {}'+str(np.std(ap_list))+"\n")

if __name__ == '__main__':
  #  ap = argparse.ArgumentParser(description='MRGNN testing for the recommendation dataset')
   # ap.add_argument('--feats-type', type=int, default=0,
   #                 help='Type of the node features used. ' +
  #                       '0 - all id vectors; ' +
 #                        '1 - all zero vector. Default is 0.')
  #  ap.add_argument('--hidden-dim', type=int, default=64, help='Dimension of the node hidden state. Default is 64.')
 #   ap.add_argument('--num-heads', type=int, default=8, help='Number of the attention heads. Default is 8.')
  #  ap.add_argument('--attn-vec-dim', type=int, default=128, help='Dimension of the attention vector. Default is 128.')
  #  ap.add_argument('--rnn-type', default='RotatE0', help='Type of the aggregator. Default is RotatE0.')
 #   ap.add_argument('--epoch', type=int, default=100, help='Number of epochs. Default is 100.')
  #  ap.add_argument('--patience', type=int, default=5, help='Patience. Default is 5.')
  #  ap.add_argument('--batch-size', type=int, default=8, help='Batch size. Default is 8.')
   # ap.add_argument('--samples', type=int, default=100, help='Number of neighbors sampled. Default is 100.')
    #ap.add_argument('--repeat', type=int, default=1, help='Repeat the training and testing for N times. Default is 1.')
   # ap.add_argument('--save-postfix', default='LastFM', help='Postfix for the saved model and result. Default is LastFM.')

  #  args = ap.parse_args()
    run_model_OURS(0, 64, 1,128, 100,3,8, 100, 1, 'ours')
 

/home/huangc/Download/anaconda/envs/workspace/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
/home/huangc/Download/anaconda/envs/workspace/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Keyword arguments ['multigraph'] are deprecated in v0.5, and can be safely removed in all cases.
  return warnings.warn(message, category=category, stacklevel=1)


Epoch 00000 | Iteration 00000 | Train_Loss 95.3323 | Time1(s) 0.2699 | Time2(s) 0.0359 | Time3(s) 0.0254
Epoch 00000 | Iteration 00100 | Train_Loss 0.6620 | Time1(s) 0.3618 | Time2(s) 0.0334 | Time3(s) 0.0231
Epoch 00000 | Iteration 00200 | Train_Loss 0.5400 | Time1(s) 0.3619 | Time2(s) 0.0335 | Time3(s) 0.0231
Epoch 00000 | Iteration 00300 | Train_Loss 0.7270 | Time1(s) 0.3624 | Time2(s) 0.0336 | Time3(s) 0.0231
Epoch 00000 | Iteration 00400 | Train_Loss 0.7684 | Time1(s) 0.3604 | Time2(s) 0.0336 | Time3(s) 0.0231
Epoch 00000 | Iteration 00500 | Train_Loss 0.4766 | Time1(s) 0.3592 | Time2(s) 0.0336 | Time3(s) 0.0231
Epoch 00000 | Iteration 00600 | Train_Loss 0.6218 | Time1(s) 0.3574 | Time2(s) 0.0336 | Time3(s) 0.0231
Epoch 00000 | Iteration 00700 | Train_Loss 0.4884 | Time1(s) 0.3568 | Time2(s) 0.0336 | Time3(s) 0.0231
Epoch 00000 | Iteration 00800 | Train_Loss 0.5757 | Time1(s) 0.3556 | Time2(s) 0.0336 | Time3(s) 0.0231
Epoch 00000 | Iteration 00900 | Train_Loss 0.6102 | Time1(s) 0.

In [29]:
num_ntype = 3
dropout_rate = 0.5
lr = 0.005
weight_decay = 0.001
hidden_dim=64
num_heads=1
attn_vec_dim=128
neighbor_samples=100
batch_size=8
#etypes_lists = [[[0, 1],[0, 2, 3, 1]],
 #               [[1, 0],[2, 3]]
etypes_lists = [[[0, 1],[0, 2, 3, 1],[4,None,5]],
                [[1, 0],[2, 3],[2, None, 3]]]
use_masks = [[True, True],
             [True, False]]
no_masks = [[False] * 3, [False] * 3]
#use_masks = [[True, True, False],
#             [True, False, True]]
#no_masks = [[False] * 3, [False] * 3]
num_mir=1296
num_disease=11783
num_gene=10116
expected_metapaths = [
   [(0, 1, 0), (0, 1, 2, 1, 0), (0,2,2,0)],
    [(1, 0, 1), (1, 2, 1), (1, 2, 2, 1)]
]
features_list = []
in_dims = []

adjlists_ua, edge_metapath_indices_list_ua, _, type_mask, train_val_test_pos_user_artist, train_val_test_neg_user_artist = load_LastFM_data()
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
for i in range(num_ntype):
    dim = (type_mask == i).sum()
    in_dims.append(dim)
    indices = np.vstack((np.arange(dim), np.arange(dim)))
    indices = torch.LongTensor(indices)
    values = torch.FloatTensor(np.ones(dim))
    features_list.append(torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device))
net = MEAHNE_lp(
            [3, 3], 6, etypes_lists, in_dims, hidden_dim, hidden_dim, num_heads, attn_vec_dim, dropout_rate)
net.to(device)
#optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
pred_idx_generator = index_generator(batch_size=batch_size, num_data=len(val_2142), shuffle=False)
net.load_state_dict(torch.load('checkpoint/checkpoint_{}.pt'.format('ours')))
net.eval()
proba_list=[]
with torch.no_grad():
    for iteration in range(pred_idx_generator.num_iterations()):
        pred_idx_batch = pred_idx_generator.next()
        pred_batch =val_2142[pred_idx_batch].tolist()
        print(pred_batch)
        pred_g_lists, pred_indices_lists, pred_idx_batch_mapped_lists,pred_pos_node_lists = parse_minibatch_LastFM(
          adjlists_ua, edge_metapath_indices_list_ua, pred_batch, device, neighbor_samples, no_masks, num_mir)
        [embedding_mir, embedding_disease], _ = net(
                    (pred_g_lists, features_list, type_mask, pred_indices_lists, pred_idx_batch_mapped_lists,pred_pos_node_lists))
        embedding_mir = embedding_mir.view(-1, 1, embedding_mir.shape[1])
        embedding_disease = embedding_disease.view(-1, embedding_disease.shape[1], 1)
        out = torch.bmm(embedding_mir, embedding_disease).flatten()
        out = out.cpu().numpy()
        proba_list.append(out)

[[0, 2142], [1, 2142], [2, 2142], [3, 2142], [4, 2142], [5, 2142], [6, 2142], [7, 2142]]
[[8, 2142], [9, 2142], [10, 2142], [11, 2142], [12, 2142], [13, 2142], [14, 2142], [15, 2142]]
[[16, 2142], [17, 2142], [18, 2142], [19, 2142], [20, 2142], [21, 2142], [22, 2142], [23, 2142]]
[[24, 2142], [25, 2142], [26, 2142], [27, 2142], [28, 2142], [29, 2142], [30, 2142], [31, 2142]]
[[32, 2142], [33, 2142], [34, 2142], [35, 2142], [36, 2142], [37, 2142], [38, 2142], [39, 2142]]
[[40, 2142], [41, 2142], [42, 2142], [43, 2142], [44, 2142], [45, 2142], [46, 2142], [47, 2142]]
[[48, 2142], [49, 2142], [50, 2142], [51, 2142], [52, 2142], [53, 2142], [54, 2142], [55, 2142]]
[[56, 2142], [57, 2142], [58, 2142], [59, 2142], [60, 2142], [61, 2142], [62, 2142], [63, 2142]]
[[64, 2142], [65, 2142], [66, 2142], [67, 2142], [68, 2142], [69, 2142], [70, 2142], [71, 2142]]
[[72, 2142], [73, 2142], [74, 2142], [75, 2142], [76, 2142], [77, 2142], [78, 2142], [79, 2142]]
[[80, 2142], [81, 2142], [82, 2142], [83

In [23]:
val_2147=[]
for i in range(1296):
    cur=[]
    cur.append(i)
    cur.append(2147)
    val_2147.append(cur)
val_2147=np.array(val_2147)
val_2147

array([[   0, 2147],
       [   1, 2147],
       [   2, 2147],
       ...,
       [1293, 2147],
       [1294, 2147],
       [1295, 2147]])

In [26]:
res_2147_list

array([[ -10.02048302, 1288.        ],
       [  -8.88030815,  760.        ],
       [  -8.87591743,   40.        ],
       ...,
       [   1.83678842,  178.        ],
       [   1.85946429,  244.        ],
       [   2.15600204,  240.        ]])

In [31]:
proba_list

[array([ 2.3979073 ,  2.6384914 ,  2.385874  , -1.1157149 ,  2.3095143 ,
         2.359257  ,  0.75443554,  0.74556327], dtype=float32),
 array([-0.169123 ,  2.1320922,  1.7925895, -1.0215013,  2.0148087,
         0.9294625,  1.7588209, -1.4702307], dtype=float32),
 array([-1.557161  ,  1.6051172 ,  1.2768508 ,  0.42727768, -1.9902754 ,
        -2.0212784 ,  0.68829894,  1.7796688 ], dtype=float32),
 array([ 2.4370763, -1.7714345,  2.1420822,  1.5901086,  2.2458584,
         0.3289292, -0.8008124, -0.8323271], dtype=float32),
 array([ 0.6409991 , -1.5067164 , -1.4473978 , -1.7148634 ,  0.26868007,
        -0.57627904, -0.98748195, -0.4290437 ], dtype=float32),
 array([-1.3708684 , -0.7397567 ,  0.23112126, -1.0679251 ,  0.46382815,
        -0.95484376, -1.037783  , -0.6369655 ], dtype=float32),
 array([ 0.6710872 ,  3.0323694 ,  0.61476684,  0.579403  ,  0.6012489 ,
         0.604111  ,  0.6238135 , -0.1450986 ], dtype=float32),
 array([-2.0566864 ,  0.40542668, -1.7267702 , -1.7844185

In [28]:
val_2142=[]
for i in range(1296):
    cur=[]
    cur.append(i)
    cur.append(2142)
    val_2142.append(cur)
val_2142=np.array(val_2142)
val_2142

array([[   0, 2142],
       [   1, 2142],
       [   2, 2142],
       ...,
       [1293, 2142],
       [1294, 2142],
       [1295, 2142]])

In [25]:
res_2147=[]
for i in range(162):
    for j in proba_list[i]:
        res_2147.append(j)
res_2147_list=[]
for i in range(1296):
    cur=[]
    cur.append(res_2147[i])
    cur.append(i)
    res_2147_list.append(cur)
res_2147_list
res_2147_list=np.array(res_2147_list)
sorted_index = sorted(list(range(len(res_2147_list))), key=lambda i : res_2147_list[i].tolist())
res_2147_list=res_2147_list[sorted_index]
res_2147_list
top_50_2147=[]
for i in range(50):
    top_50_2147.append(res_2147_list[1295-i][1])
top_50_2147

[240.0,
 244.0,
 178.0,
 246.0,
 362.0,
 392.0,
 252.0,
 389.0,
 400.0,
 283.0,
 430.0,
 421.0,
 406.0,
 95.0,
 377.0,
 176.0,
 452.0,
 49.0,
 632.0,
 364.0,
 436.0,
 267.0,
 391.0,
 1261.0,
 303.0,
 1.0,
 405.0,
 434.0,
 401.0,
 337.0,
 463.0,
 245.0,
 837.0,
 968.0,
 1258.0,
 284.0,
 416.0,
 835.0,
 247.0,
 1257.0,
 456.0,
 1256.0,
 431.0,
 184.0,
 187.0,
 455.0,
 1255.0,
 1203.0,
 353.0,
 196.0]

In [30]:
res_2142=[]
for i in range(162):
    for j in proba_list[i]:
        res_2142.append(j)
res_2142_list=[]
for i in range(1296):
    cur=[]
    cur.append(res_2142[i])
    cur.append(i)
    res_2142_list.append(cur)
res_2142_list
res_2142_list=np.array(res_2142_list)
sorted_index = sorted(list(range(len(res_2142_list))), key=lambda i : res_2142_list[i].tolist())
res_2142_list=res_2142_list[sorted_index]
res_2142_list
top_50_2142=[]
for i in range(50):
    top_50_2142.append(res_2142_list[1295-i][1])
top_50_2142

[362.0,
 240.0,
 364.0,
 246.0,
 244.0,
 178.0,
 252.0,
 389.0,
 202.0,
 95.0,
 49.0,
 219.0,
 230.0,
 391.0,
 392.0,
 196.0,
 561.0,
 218.0,
 337.0,
 563.0,
 412.0,
 409.0,
 552.0,
 377.0,
 313.0,
 206.0,
 94.0,
 303.0,
 204.0,
 89.0,
 436.0,
 65.0,
 763.0,
 434.0,
 212.0,
 414.0,
 837.0,
 400.0,
 310.0,
 340.0,
 1.0,
 200.0,
 209.0,
 370.0,
 353.0,
 452.0,
 430.0,
 351.0,
 358.0,
 538.0]

In [32]:
res_2142_list

array([[  -2.45583487, 1288.        ],
       [  -2.26642895,  394.        ],
       [  -2.2421236 ,  418.        ],
       ...,
       [   3.4009707 ,  364.        ],
       [   3.52465439,  240.        ],
       [   3.91114902,  362.        ]])